# geofetch tutorial for processed data

The [GSE185701 data set](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE185701) has about 355 Mb of processed data that contains 57 Supplementary files, so it's a quick download for a test case. Let's take a quick peek at the geofetch version:

In [1]:
geofetch --version

geofetch 0.9.0


To see your CLI options, invoke `geofetch -h`:

In [2]:
geofetch -h

usage: geofetch [-h] [-V] -i INPUT [-n NAME] [-m METADATA_ROOT]
                [-u METADATA_FOLDER] [--just-metadata] [-r]
                [--config-template CONFIG_TEMPLATE]
                [--pipeline_samples PIPELINE_SAMPLES]
                [--pipeline_project PIPELINE_PROJECT] [-k SKIP] [--acc-anno]
                [--discard-soft] [--const-limit-project CONST_LIMIT_PROJECT]
                [--const-limit-discard CONST_LIMIT_DISCARD]
                [--attr-limit-truncate ATTR_LIMIT_TRUNCATE] [-p]
                [--data-source {all,samples,series}] [--filter FILTER]
                [--filter-size FILTER_SIZE] [-g GEO_FOLDER] [-x]
                [-b BAM_FOLDER] [-f FQ_FOLDER] [--use-key-subset] [--silent]
                [--verbosity V] [--logdev]

Automatic GEO and SRA data downloader

optional arguments:
  -h, --help            show this help message and exit
  -V, --version         show program's version number and exit
  -i INPUT, --input INPUT
                        requir

Calling geofetch will do 4 tasks: 

1. download all or filtered processed files from `GSE#####` into your geo folder.
2. download all metadata from GEO and store in your metadata folder.
2. produce a PEP-compatible sample table, `PROJECT_NAME_sample_processed.csv` and `PROJECT_NAME_series_processed.csv`, in your metadata folder.
3. produce a PEP-compatible project configuration file, `PROJECT_NAME_sample_processed.yaml` and `PROJECT_NAME_series_processed.yaml`, in your metadata folder.

Complete details about geofetch outputs is cataloged in the [metadata outputs reference](metadata_output.md).

## Download the data

First, create the metadata for processed data (by adding --processed and --just-metadata):

In [3]:
geofetch -i GSE185701 --processed -n bright_test --just-metadata

Metadata folder: /home/bnt4me/Virginia/repos/geof2/geofetch/docs_jupyter/bright_test
Trying GSE185701 (not a file) as accession...
Skipped 0 accessions. Starting now.
Processing accession 1 of 1: 'GSE185701'
--2022-06-10 14:43:11--  https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?targ=gse&acc=GSE185701&form=text&view=full
Resolving www.ncbi.nlm.nih.gov (www.ncbi.nlm.nih.gov)... 2607:f220:41e:4290::110, 130.14.29.110
Connecting to www.ncbi.nlm.nih.gov (www.ncbi.nlm.nih.gov)|2607:f220:41e:4290::110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [geo/text]
Saving to: ‘/home/bnt4me/Virginia/repos/geof2/geofetch/docs_jupyter/bright_test/GSE185701_GSE.soft’

/home/bnt4me/Virgin     [ <=>                ]   2.82K  --.-KB/s    in 0s      

2022-06-10 14:43:12 (207 MB/s) - ‘/home/bnt4me/Virginia/repos/geof2/geofetch/docs_jupyter/bright_test/GSE185701_GSE.soft’ saved [2885]

--2022-06-10 14:43:12--  https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?targ=gsm&acc

In [4]:
ls bright_test

GSE185701_file_list.txt  GSE185701_GSM.soft     GSE185701_samples.yaml
GSE185701_GSE.soft       GSE185701_samples.csv


The `.soft` files are the direct output from GEO, which contain all the metadata as stored by GEO, for both the experiment (`_GSE`) and for the individual samples (`_GSM`). Geofetch also produces a `csv` file with the SRA metadata. The filtered version (ending in `_filt`) would contain only the specified subset of the samples if we didn't request them all, but in this case, since we only gave an accession, it is identical to the complete file. Additionally, file_list.txt is downloaded, that contains information about size, type and creation date of all sample files.

Finally, there are the 2 files that make up the PEP: the `_config.yaml` file and the `_annotation.csv` file (for samples and series). Let's see what's in these files now.

In [10]:
cat bright_test/GSE185701_samples.yaml

# Autogenerated by geofetch

pep_version: 2.1.0
project_name: GSE185701
sample_table: GSE185701_samples.csv

sample_modifiers:
  append:
    output_file_path: FILES
    Sample_growth_protocol_ch1: Huh 7 was cultured in Dulbecco’s modified Eagle’s medium (DMEM) (Invitrogen, Carlsbad, CA, USA) containing 10% fetal bovine serum (FBS) (HyClone, Logan, UT, USA) and antibiotics (penicillin and streptomycin, Invitrogen) at 37 °C in 5% CO2.
    
  derive:
    attributes: [output_file_path]
    sources:
      FILES: /{GSE}/{file}





There are few important things to note in this file: 
* First, see in the PEP that `sample_table` points to the csv file produced by geofetch. 
* Second: output_file_path is location of all the files. 
* Third: sample_modifier Sample_growth_protocol_ch1 is constant sample character and is larger then 50 characters so it is deleted from csv file. For large project it can significantly reduced size of the metadata

Now let's look at the first 100 characters of the csv file:

In [11]:
cut -c -100 bright_test/GSE185701_samples.csv

GSE,Sample_title,Sample_geo_accession,Sample_status,Sample_submission_date,Sample_last_update_date,S
GSE185701,Huh7_siNC_H3K27ac,GSM5621756,Public on Mar 01 2022,Oct 12 2021,Mar 03 2022,SRA,1,Huh 7,Hom
GSE185701,Huh7_siNC_H3K27ac,GSM5621756,Public on Mar 01 2022,Oct 12 2021,Mar 03 2022,SRA,1,Huh 7,Hom
GSE185701,Huh7_siDHX37_H3K27ac,GSM5621758,Public on Mar 01 2022,Oct 12 2021,Mar 03 2022,SRA,1,Huh 7,
GSE185701,Huh7_siDHX37_H3K27ac,GSM5621758,Public on Mar 01 2022,Oct 12 2021,Mar 03 2022,SRA,1,Huh 7,
GSE185701,Huh7_DHX37,GSM5621760,Public on Mar 01 2022,Oct 12 2021,Mar 03 2022,SRA,1,Huh 7,Homo sapie
GSE185701,Huh7_DHX37,GSM5621760,Public on Mar 01 2022,Oct 12 2021,Mar 03 2022,SRA,1,Huh 7,Homo sapie
GSE185701,Huh7_PLRG1,GSM5621761,Public on Mar 01 2022,Oct 12 2021,Mar 03 2022,SRA,1,Huh 7,Homo sapie
GSE185701,Huh7_PLRG1,GSM5621761,Public on Mar 01 2022,Oct 12 2021,Mar 03 2022,SRA,1,Huh 7,Homo sapie


Now let's download the actual data. This time we will will be downloading data from the [GSE185701 data set](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE185701) .
Let's additionally add few arguments:
- _geo-folder_ (required) - path to the location where processed files have to be saved
- _filter_ argument, to download only _bed_ files  (--filter ".Bed.gz$") 
- _data-source_ argument, to download files only from sample location (--data-source samples)

In [12]:
geofetch -i GSE185701 --processed -n bright_test --filter ".bed.gz$" --data-source samples \
--geo-folder /home/bnt4me/Virginia/for_docs/geo

Metadata folder: /home/bnt4me/Virginia/repos/geof2/geofetch/docs_jupyter
Trying GSE185701 (not a file) as accession...
Skipped 0 accessions. Starting now.
Processing accession 1 of 1: 'GSE185701'
--2022-06-10 14:53:15--  https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?targ=gse&acc=GSE185701&form=text&view=full
Resolving www.ncbi.nlm.nih.gov (www.ncbi.nlm.nih.gov)... 2607:f220:41e:4290::110, 130.14.29.110
Connecting to www.ncbi.nlm.nih.gov (www.ncbi.nlm.nih.gov)|2607:f220:41e:4290::110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [geo/text]
Saving to: ‘/home/bnt4me/Virginia/repos/geof2/geofetch/docs_jupyter/GSE185701_GSE.soft’

/home/bnt4me/Virgin     [ <=>                ]   2.82K  --.-KB/s    in 0s      

2022-06-10 14:53:16 (898 MB/s) - ‘/home/bnt4me/Virginia/repos/geof2/geofetch/docs_jupyter/GSE185701_GSE.soft’ saved [2885]

--2022-06-10 14:53:16--  https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?targ=gsm&acc=GSE185701&form=text&view=full
Resol

Now lets list the folder to see what data is there. And let's see what's in pep files now.

In [13]:
ls /home/bnt4me/Virginia/for_docs/geo/GSE185701

GSM5621756_ChIPseq_Huh7_siNC_H3K27ac_summits.bed.gz
GSM5621758_ChIPseq_Huh7_siDHX37_H3K27ac_summits.bed.gz
GSM5621760_CUTTag_Huh7_DHX37_summits.bed.gz
GSM5621761_CUTTag_Huh7_PLRG1_summits.bed.gz


In [10]:
cut -c -100 bright_test/bright_test_annotation_sample_processed.csv

GSE,Sample_title,Sample_geo_accession,Sample_status,Sample_submission_date,Sample_last_update_date,S
GSE185701,Huh7_siNC_H3K27ac,GSM5621756,Public on Mar 01 2022,Oct 12 2021,Mar 03 2022,SRA,1,Huh 7,Hom
GSE185701,Huh7_siDHX37_H3K27ac,GSM5621758,Public on Mar 01 2022,Oct 12 2021,Mar 03 2022,SRA,1,Huh 7,
GSE185701,Huh7_DHX37,GSM5621760,Public on Mar 01 2022,Oct 12 2021,Mar 03 2022,SRA,1,Huh 7,Homo sapie
GSE185701,Huh7_PLRG1,GSM5621761,Public on Mar 01 2022,Oct 12 2021,Mar 03 2022,SRA,1,Huh 7,Homo sapie


In [11]:
cat bright_test/bright_test_annotation_sample_processed.yaml

pep_version: 2.0.0
project_name: bright_test
sample_table: /home/bnt4me/Virginia/repos/geof2/geofetch/docs_jupyter/bright_test/bright_test_annotation_sample_processed.csv

sample_modifiers:
  append:
    output_file_path: FILES
  derive:
    attributes: [output_file_path]
    sources:
      FILES: /home/bnt4me/Virginia/for_docs/geo/{SRA}/{sample_name}




Now we have easy access to this data by using [peppy](http://peppy.databio.org/en/latest/) package in python or [pepr](https://code.databio.org/pepr/) in r in further analysis 